## Turning Addresses into Latitude and Longitude in Chargepoint
In this Notebook we will be working with the Chargepoint data set. Right now the Chargepoint data only has the addresses of the stations, however, we want the latitudes and longitudes too. The following code takes the addresses column and uses google to make a new column with latitude and longitude.

Below we are loading libraries into the notebook. This step makes it so some of the code we write will work! - You wont ever need to touch this code.

In [1]:
library(ggmap)
library(rworldmap)
library(dplyr)

Loading required package: ggplot2
Loading required package: sp
### Welcome to rworldmap ###
For a short introduction type : 	 vignette('rworldmap')

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
Chargepoint <- read.csv("Chargepoint.csv", stringsAsFactors = FALSE)

## Manipulating the Addresses column

We are going to use a program that takes the addresses in the dataset and turns it into latitude and longitudes. The program we are using requires us to have the address line in one column; right now it is in three.
##### *Current Format:* |1650 Ala Moana Blvd|Honolulu|Hawaii|
##### *We Want it to be:* |1650 Ala Moana Blvd, Honolulu, Hawaii|

In [3]:
# To break down the code below: We are pasting together three columns (Address.1, City, State) 
#in the Chargepoint data, separating them by a comma (sep = ",") and saving them into a new dataset 
#called "addresses"
addresses <- paste(Chargepoint$Address.1, Chargepoint$City, Chargepoint$State, sep = ",")

Below we are putting the "addresses" dataset back onto the Chargepoint data set, which is kind of like adding a new column called "addresses" to the Chargepoint data.

In [4]:
Chargepoint <- (cbind(Chargepoint, addresses))
Chargepoint$lon <- NA
Chargepoint$lat <- NA

In the below code we are filtering out the addresses that occur more than once. We are doing this because sometimes the program that uses the addresses gets confused looking at the same address multiple times. So for now we are taking it out, we will eventually put it back in!

In [5]:
geocoded <- data.frame(stringsAsFactors = FALSE)
# getting uniques in the Chargepoint_address so maybe it wont error
CP_address_unq <- distinct(Chargepoint, Address.1, Station.Name, addresses, .keep_all = TRUE)
#turning addresses from integer to character
CP_address_unq$addresses <- as.character(CP_address_unq$addresses)

## From Address to Latitude & Longitude
The below code it where the magic happens! We send the addresses to google, google generates the latitude and longitude and then puts it in a new column in the dataset.

In [6]:
# Loop through the addresses to get the latitude and longitude of each address and add it to the
# origAddress data frame in new columns lat and lon
# I had to run it a few times until I stopped getting NA's!
while (sum(is.na(CP_address_unq$lon)))
{
    for(i in 1:nrow(CP_address_unq))
    {if (is.na(CP_address_unq$lon[i])){
      result <- geocode(CP_address_unq$addresses[i], output = "latlona", source = "google")
      CP_address_unq$lon[i] <- as.numeric(result[1])
      CP_address_unq$lat[i] <- as.numeric(result[2])}
    }
}

Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=2520%20Dole%20St,Honolulu,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=2520%20Dole%20St,Honolulu,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=925%20Bethel%20St,Honolulu,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=1888-1898%20Wili%20Pa%20Loop,Wailuku,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=94-815%20Lumiaina%20St,Waipahu,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=459%20Patterson%20Rd,Honolulu,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=459%20Patterson%20Rd,Honolulu,Hawaii&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=459%20Patters

In [7]:
sum(is.na(CP_address_unq$lat))

[1] 0

- Remember above when we filtered out the addresses that had multiples? Now we are putting them back in!
- We are also getting rid of a few columns that we dont need.

In [8]:
#taking out everything except addresses and lat/long
Chargepointgeocode = subset(CP_address_unq, select = -c(Postal.Code,County,Country))

Charge_point_final <- left_join(Chargepoint,Chargepointgeocode, by = "addresses")
#View(jointest)

Warning message:
“Column `addresses` joining factor and character vector, coercing into character vector”

In [9]:
head(Charge_point_final)

X.x,Station.Name.x,Org.Name.x,Address.1.x,City.x,State.x,Postal.Code,County,Country,No..of.Ports.x,⋯,Station.Name.y,Org.Name.y,Address.1.y,City.y,State.y,No..of.Ports.y,Station.Activation.Type.y,Customer.Category.y,lon.y,lat.y
1,ZONE 20 / ZONE 20 #2,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,96822,Honolulu County,United States,2,⋯,ZONE 20 / ZONE 20 #2,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,2,Public,Education,-157.8175,21.29571
1,ZONE 20 / ZONE 20 #2,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,96822,Honolulu County,United States,2,⋯,ZONE 20 / ZONE 20 #1,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,2,Public,Education,-157.8175,21.29571
2,ZONE 20 / ZONE 20 #1,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,96822,Honolulu County,United States,2,⋯,ZONE 20 / ZONE 20 #2,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,2,Public,Education,-157.8175,21.29571
2,ZONE 20 / ZONE 20 #1,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,96822,Honolulu County,United States,2,⋯,ZONE 20 / ZONE 20 #1,University of Hawaii at Manoa,2520 Dole St,Honolulu,Hawaii,2,Public,Education,-157.8175,21.29571
3,WWW.HIEV.US / HIEV DEMO 001,HIEV DEMO UNITS,925 Bethel St,Honolulu,Hawaii,96813,Honolulu County,United States,1,⋯,WWW.HIEV.US / HIEV DEMO 001,HIEV DEMO UNITS,925 Bethel St,Honolulu,Hawaii,1,Public,Demo Unit,-157.8627,21.30978
4,WAILUKU BRANCH / MCFU WAILUKU 1,Maui County Federal Credit Union,1888-1898 Wili Pa Loop,Wailuku,Hawaii,96793,Maui County,United States,2,⋯,WAILUKU BRANCH / MCFU WAILUKU 1,Maui County Federal Credit Union,1888-1898 Wili Pa Loop,Wailuku,Hawaii,2,Public,Workplace,-156.4995,20.89431


## Behold

Think of the below code as a SAVE AS. We now have all of our work in a new file called "Chargepoint_Goecode"

In [10]:
write.csv(Charge_point_final, "Geocoded_Chargepoint.csv", row.names=FALSE)